# Instacart Grocery Basket Data Analysis: Final Report Bonus
Script by Adilya Zaripova

## Addressing Missing Product Data in Instacart Analysis

In our examination of the Instacart final project, our analysis solely relies on the fully integrated dataset. However, upon performing a comprehensive full outer join of the products and orders datasets, a substantial number of mismatched records came to light.
As a part of the exercise, I've deleted 16 rows of data from the products data where product names were missing. As a result, after performing a full outer join, I encountered instances of product IDs that do not align with any corresponding products in the provided product dataset. There are also instances where distinct products share identical IDs.

Scenario: I imagime flagging this issues to Instacart management and receiving a request to fix the data. 

1. I fix product IDs in products data set and replace them with updated values.
2. I receive information about the products missing from the products dataset. Using this information I create a new dataframe and append it to the original products data.
3. I confirm that the data is now fully merged and there are no missing values with the exeption of the order information for the products that have never been ordered.

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import random

In [2]:
path=r'C:\Users\frauz\Documents\Python Projects\08-2023 Instacart Basket Analysis\02.Data'

### Identical product IDs

In [3]:
# Importing the products dataset

df_products=pd.read_csv(os.path.join(path, 'Prepared data', 'products_checked.csv'), index_col=False)
df_products.drop(columns='Unnamed: 0', inplace=True)

In [4]:
df_products.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


In [5]:
# checking the number of uniqe/distinct values in each column

df_products.nunique(axis=0)

product_id       49670
product_name     49672
aisle_id           134
department_id       21
prices             242
dtype: int64

In [6]:
df_zero=df_products[df_products['product_name']==0]
df_zero.shape

(0, 5)

#### I see that the number of unique product IDs doesn't match the number of records - 49672. That means that we have different products with the same product ID. I'm going to keep investigating.

In [6]:
# Looking for product IDs that appear more than once

df_products['product_id'].value_counts()

26520    2
6800     2
1        1
33136    1
33127    1
        ..
16576    1
16577    1
16578    1
16579    1
49688    1
Name: product_id, Length: 49670, dtype: int64

In [7]:
# Let's look at products with identical IDs

df_products[df_products['product_id'].isin([26520,6800])]

,product_id,product_name,aisle_id,department_id,prices
6784,6800,Revive Zero Vitamin Water,64,7,6.4
6785,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26504,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26505,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


#### Clearly, we have an issue with product IDs: there are different products with the same IDs in the products dataset.

### Missing products

In [8]:
# Importing the complete data created by full outer join of orders and products datasets

df_full=pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_full.pkl'))

In [9]:
df_full.isnull().sum()

order_id                       11
product_id                      0
add_to_cart_order              11
reordered                      11
user_id                        11
order_number                   11
order_day_of_the_week          11
order_hour_of_day              11
days_since_prior_order    2078113
product_name                30200
aisle_id                    30200
department_id               30200
prices                      30200
_merge                          0
dtype: int64

Missing values in 'order_id' columns indicate that 11 products have never been ordered. 

In [10]:
# Creating a subset of data with missing values in 'product_name' column.

df_missing=df_full[df_full['product_name'].isnull()]

In [11]:
df_missing.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_day_of_the_week,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,prices,_merge
6024648,44.0,2240,9.0,0.0,183833.0,3.0,6.0,17.0,1.0,NaN,NaN,NaN,NaN,left_only
6024649,7889.0,2240,5.0,0.0,59913.0,35.0,1.0,12.0,2.0,NaN,NaN,NaN,NaN,left_only
6024650,8008.0,2240,5.0,1.0,167797.0,13.0,5.0,1.0,12.0,NaN,NaN,NaN,NaN,left_only
6024651,10548.0,2240,18.0,0.0,61148.0,33.0,4.0,9.0,11.0,NaN,NaN,NaN,NaN,left_only
6024652,12814.0,2240,14.0,1.0,57530.0,29.0,3.0,10.0,4.0,NaN,NaN,NaN,NaN,left_only


In [12]:
df_missing.shape

(30200, 14)

In [13]:
# Counting the records by order ID 

df_missing_id=df_missing['product_id'].value_counts().rename_axis('id').reset_index(name='count')

df_missing_id

,id,count
0,1511,13397
1,34,6536
2,116,4359
3,6799,1978
4,4790,1804
5,2240,1689
6,262,179
7,3230,55
8,26519,51
9,1780,39


In [14]:
# Creating a list of IDs with no product inormation

list_id=df_missing_id['id'].to_list()
print(list_id)

[1511, 34, 116, 6799, 4790, 2240, 262, 3230, 26519, 1780, 2586, 69, 525, 4283, 40440, 3736, 3159, 38183]


##### Scenario: I send the list of of ids to the namager and receive the information about the missing products. I also learn that two products were accidentally recorded with a wrond ID.  

Revive Zero Vitamin Water has an ID 1511  
Cheese Shredded Sharp Cheddar Reduced Fat 2% has an ID 34


In [15]:
# Removing existing products ID from the missing IDs list

list_id.remove(1511)
list_id.remove(34)
print(list_id)

[116, 6799, 4790, 2240, 262, 3230, 26519, 1780, 2586, 69, 525, 4283, 40440, 3736, 3159, 38183]


In [16]:
# Finding the correct headers for the new dataframe

df_products.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')

In [17]:
# Creating a dataframe using information about the missing products
x=random.sample(range(1,134),len(list_id))
y=random.sample(range(1,21),len(list_id))
names=['Savory Crunch Mix',
       'Yummy-yummy food',
        'Creamy Hazelnut Delight',
        'Zesty Mango Tango',
        'Golden Honey Wheat',
        'Spicy Jalapeno Fusion',
        'Choco-Coconut Bliss',
        'Classic Roast Blend',
        'Crispy Onion Medley',
        'Sweet Berry Burst',
        'Herb-Infused Olive Elixir',
        'Hearty Vegetable Medley',
        'Citrus Burst Quencher',
        'Gourmet Truffle Indulgence',
        'Smoky BBQ Temptation',
        'Luscious Caramel Swirl']
prices=np.random.uniform(low=1.0, high=25.0, size=len(list_id))

print(x)
print(y)
print(prices)
print(len(names))

In [19]:
# Creating a new dataframe

df_missing_products=pd.DataFrame({'product_id':list_id,
                                 'product_name':names,
                                 'aisle_id':x,
                                 'department_id':y,
                                 'prices':prices.round(1)})
 

In [20]:
# Now I have a dataframe of missing products
df_missing_products

,product_id,product_name,aisle_id,department_id,prices
0,116,Savory Crunch Mix,87,10,7.1
1,6799,Yummy-yummy food,119,15,16.0
2,4790,Creamy Hazelnut Delight,42,17,12.7
3,2240,Zesty Mango Tango,8,4,5.1
4,262,Golden Honey Wheat,66,18,7.7
5,3230,Spicy Jalapeno Fusion,5,16,1.9
6,26519,Choco-Coconut Bliss,74,13,23.5
7,1780,Classic Roast Blend,15,1,12.8
8,2586,Crispy Onion Medley,38,2,3.3
9,69,Sweet Berry Burst,6,5,7.3


In [21]:
# Replacing ID of the products recorded with a mistake

df_products.loc[df_products['product_name']=='Revive Zero Vitamin Water', 'product_id']=1511

df_products.loc[df_products['product_name']=='Cheese Shredded Sharp Cheddar Reduced Fat 2%', 'product_id']=34

df_products[df_products['product_id'].isin([1511,34])]

,product_id,product_name,aisle_id,department_id,prices
6784,1511,Revive Zero Vitamin Water,64,7,6.4
26505,34,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [22]:
# Concatanating the two dataframes

df_products_updated=pd.concat([df_products,df_missing_products])

In [23]:
df_products_updated

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3
...,...,...,...,...,...
11,4283,Hearty Vegetable Medley,116,8,4.2
12,40440,Citrus Burst Quencher,36,20,3.3
13,3736,Gourmet Truffle Indulgence,95,3,7.6
14,3159,Smoky BBQ Temptation,50,12,2.4


In [24]:
# Controlling the result

len(df_products)+len(df_missing_products)

49688

In [25]:
# Now we can merge the updated products dataframe with the orders dataset. First, importing the dataset

df_order=pd.read_pickle(os.path.join(path,'Prepared Data','orders_products_combined.pkl'))

In [26]:
df_order.drop(columns='_merge', inplace=True)

In [27]:
# Merging the orders and products data using outer join

df_merge_outer=df_order.merge(df_products_updated, on='product_id', how='outer', indicator=True)

In [28]:
df_merge_outer['_merge'].value_counts()

both          32434489
right_only          11
left_only            0
Name: _merge, dtype: int64

#### Now the data is fully merged. The 11 records missing from the orders dataframe should be the products that have never been ordered and don't have a record.

In [29]:
df_never_ordered=df_merge_outer[df_merge_outer['order_id'].isnull()]

In [31]:
#Here is a list of products that haven't been ordered yet

df_never_ordered[['product_id','product_name']]

,product_id,product_name
32434489,3630,Protein Granola Apple Crisp
32434490,3718,Wasabi Cheddar Spreadable Cheese
32434491,7045,Unpeeled Apricot Halves in Heavy Syrup
32434492,25383,Chocolate Go Bites
32434493,27499,Non-Dairy Coconut Seven Layer Bar
32434494,36233,Water With Electrolytes
32434495,37703,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...
32434496,43725,Sweetart Jelly Beans
32434497,45971,12 Inch Taper Candle White
32434498,46625,Single Barrel Kentucky Straight Bourbon Whiskey


In [32]:
# Exporting the complete updated dataframe

df_merge_outer.to_pickle(os.path.join(path, 'Prepared Data', 'fully_updated_data_bonus.pkl'))